In [1]:
import pandas as pd
import plotly.express as px
import nbformat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score




In [3]:
DinningPred = pd.read_excel("DinningData.xlsx", sheet_name='Data for 2025 pred')


In [5]:
DinningPred.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DEP          285 non-null    int64  
 1   Semester     285 non-null    int64  
 2   Month        285 non-null    int64  
 3   Year         285 non-null    int64  
 4   Hours        240 non-null    float64
 5   Money Spent  285 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 13.5 KB


In [6]:
missing_values = DinningPred.isnull().sum()
missing_values


DEP             0
Semester        0
Month           0
Year            0
Hours          45
Money Spent     0
dtype: int64

In [7]:
print(DinningPred.columns)

Index(['DEP', 'Semester', 'Month', 'Year', 'Hours', 'Money Spent'], dtype='object')


In [8]:
def linear_regression_impute(df, target_column, feature_columns):
    # Separate the data into rows with and without missing values
    df_missing = df[df[target_column].isnull()]
    df_not_missing = df.dropna(subset=[target_column])

    # If there are no missing values, return the original dataframe
    if df_missing.empty:
        print(f"No missing values for {target_column}. Skipping imputation.")
        return df

    # Create training and test sets for the model
    X_train = df_not_missing[feature_columns]
    y_train = df_not_missing[target_column]

    # Create and fit a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict missing values using the model
    X_missing = df_missing[feature_columns]
    predicted_values = model.predict(X_missing)

    # Create a dataset with imputed missing values
    df_filled = df.copy()
    df_filled.loc[df_missing.index, target_column] = predicted_values

    return df_filled

In [9]:
target_columns_to_impute = ['Hours']
feature_columns_for_imputation = ['Year', 'Month', 'DEP', 'Semester']
linear_data = DinningPred.copy()


In [10]:
linear_data

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.98,31844.8350
1,1,1,10,2020,4722.98,38964.5850
2,1,1,11,2020,2842.68,23452.1100
3,1,1,12,2020,1348.50,11125.1250
4,1,2,1,2021,2247.99,18545.9175
...,...,...,...,...,...,...
280,5,2,1,2025,NaN,0.0000
281,5,3,2,2025,NaN,0.0000
282,5,3,3,2025,NaN,0.0000
283,5,3,4,2025,NaN,0.0000


In [11]:
for target_column in target_columns_to_impute:
    linear_data = linear_regression_impute(linear_data, target_column, feature_columns_for_imputation)

In [12]:
linear_data

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.980000,31844.8350
1,1,1,10,2020,4722.980000,38964.5850
2,1,1,11,2020,2842.680000,23452.1100
3,1,1,12,2020,1348.500000,11125.1250
4,1,2,1,2021,2247.990000,18545.9175
...,...,...,...,...,...,...
280,5,2,1,2025,26.699803,0.0000
281,5,3,2,2025,-249.677984,0.0000
282,5,3,3,2025,-287.349391,0.0000
283,5,3,4,2025,-325.020798,0.0000


In [13]:
missing_values = linear_data.isnull().sum()
missing_values

DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64

In [15]:
df_2022 = linear_data[linear_data['Year'] == 2025]
df_2022


,DEP,Semester,Month,Year,Hours,Money Spent
52,1,2,1,2025,1111.948066,0.0
53,1,3,2,2025,835.570280,0.0
54,1,3,3,2025,797.898872,0.0
55,1,3,4,2025,760.227465,0.0
56,1,3,5,2025,722.556058,0.0
109,2,2,1,2025,840.636001,0.0
110,2,3,2,2025,564.258214,0.0
111,2,3,3,2025,526.586806,0.0
112,2,3,4,2025,488.915399,0.0
113,2,3,5,2025,451.243992,0.0


In [16]:
linear_data = linear_data.applymap(lambda x: max(0, x))


/var/folders/b5/c2s8x6hx5bz57tz7p2pf4f2h0000gn/T/ipykernel_73428/2910141043.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  linear_data = linear_data.applymap(lambda x: max(0, x))


In [1]:
df_2024 = linear_data[linear_data['Year'] == 2024]
df_2024

NameError: name 'linear_data' is not defined

In [22]:
df_2025 = linear_data[linear_data['Year'] == 2025]
df_2025

,DEP,Semester,Month,Year,Hours,Money Spent
52,1,2,1,2025,1111.948066,0.0
53,1,3,2,2025,835.570280,0.0
54,1,3,3,2025,797.898872,0.0
55,1,3,4,2025,760.227465,0.0
56,1,3,5,2025,722.556058,0.0
109,2,2,1,2025,840.636001,0.0
110,2,3,2,2025,564.258214,0.0
111,2,3,3,2025,526.586806,0.0
112,2,3,4,2025,488.915399,0.0
113,2,3,5,2025,451.243992,0.0


In [18]:
PredictedData = linear_data.copy()


In [19]:
PredictedData

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.980000,31844.8350
1,1,1,10,2020,4722.980000,38964.5850
2,1,1,11,2020,2842.680000,23452.1100
3,1,1,12,2020,1348.500000,11125.1250
4,1,2,1,2021,2247.990000,18545.9175
...,...,...,...,...,...,...
280,5,2,1,2025,26.699803,0.0000
281,5,3,2,2025,0.000000,0.0000
282,5,3,3,2025,0.000000,0.0000
283,5,3,4,2025,0.000000,0.0000


In [20]:
PredictedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DEP          285 non-null    int64  
 1   Semester     285 non-null    int64  
 2   Month        285 non-null    int64  
 3   Year         285 non-null    int64  
 4   Hours        285 non-null    float64
 5   Money Spent  285 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 13.5 KB


In [21]:
missing_values = PredictedData.isnull().sum()
missing_values

DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64